Copyright (c) 2021, 2022 Oracle and/or its affiliates.
Licensed under the Universal Permissive License v 1.0 as shown at https://oss.oracle.com/licenses/upl/

## Getting Connected

### Configuring Your Connection

There are two IAM policy validation options in `ocifs`. The first option is using an identity policy, which is a configuration file. This is the most commonly used policy, and is the default in this documentation. 

In [1]:
from ocifs import OCIFileSystem

fs = OCIFileSystem(config="~/.oci/config", profile="DEFAULT")

Using Pandas and Dask are just as easy:

In [ ]:
import pandas as pd

pd.read_csv("oci://bucket@namespace/path/file.csv", 
            storage_options={"config": "~/.oci/config", "profile": "DEFAULT"})

In [ ]:
from dask import dataframe as dd

dd.read_csv("oci://bucket@namespace/path/file.csv", 
            storage_options={"config": "~/.oci/config", "profile": "DEFAULT"})

### Using Environment Variables
From version 1.1.0, users can provide default authentication configuration details as enviornment variables.
The following environment variables are inspected:

* `OCIFS_IAM_TYPE`: which can be any of: ["api_key", "resource_principal", "instance_principal", "unknown_signer"]
* `OCIFS_CONFIG_LOCATION`: (optional) will be referenced in the case of "api_key", but defaults to the default config location provided by the oci sdk
* `OCIFS_CONFIG_PROFILE`: (optional) will be referenced in the case of "api_key", but defaults to the default config profile provided by the oci sdk


In [ ]:
import os

os.environ['OCIFS_IAM_TYPE'] = "api_key"
fs = OCIFileSystem()

Note, the order of precedence for authentication is: signer arg, config arg, environment variables, then ocifs will attempt to set up Resource Principal, as exemplified below.

### Resource Principal

The second policy option is using a resource principal. This policy only works if you're operating within a valid OCI resource, such as an [OCI Data Science notebook session](https://www.oracle.com/data-science/cloud-infrastructure-data-science.html). With this option, your resource token path is set by [global OCI signing variables](https://oracle-cloud-infrastructure-python-sdk.readthedocs.io/en/latest/api/signing.html).

In [ ]:
fs = OCIFileSystem()

And with pandas or dask:

In [ ]:
pd.read_csv("oci://bucket@namespace/path/file.csv")

In [ ]:
dd.read_csv("oci://bucket@namespace/path/file.csv")

### Connecting Using a Signer
Any signer can be passed in using the signer argument.

In [ ]:
resource_principal_signer = oci.auth.signer.get_resource_principals_signer()
fs_rp = OCIFileSystem(signer=resource_principal_signer)

In [ ]:
instance_principal_signer = oci.auth.signers.InstancePrincipalsSecurityTokenSigner()
fs_ip = OCIFileSystem(signer=instance_principal_signer)

And with pandas or dask:

In [ ]:
pd.read_csv("oci://bucket@namespace/path/file.csv", 
            storage_options={"signer": resource_principal_signer})

dd.read_csv("oci://bucket@namespace/path/file.csv",
            storage_options={"signer": instance_principal_signer})

### Connecting to a Different Region

Each filesystem instance has a home region and won't operate outside of that region. The home region defaults to the region of the IAM policy. With a configuration policy, it is `region`. With a resource principal, the region is derived from the `OCI_REGION_METADATA` environment variable. 

The `OCIFileSystem` delegates this region set up to the [Object Storage Client __init__ method](https://oracle-cloud-infrastructure-python-sdk.readthedocs.io/en/latest/api/object_storage/client/oci.object_storage.ObjectStorageClient.html#oci.object_storage.ObjectStorageClient.__init__) in the OCI Python SDK. The `region` argument accepts any [valid region identifier](https://docs.oracle.com/en-us/iaas/Content/General/Concepts/regions.htm) and constructs the corresponding service endpoint for the Object Storage Client. The following cell is an example of connecting to the sydney region.

In [ ]:
fs_sydney = OCIFileSystem(config="~/.oci/config", region="ap-sydney-1")

Using Pandas or Dask:

In [2]:
df.to_csv("oci://bucket@namespace/path/file.csv", 
          storage_options = {"config": "~/.oci/config", region="ap-sydney-1"})

In [ ]:
ddf.to_csv("oci://bucket@namespace/path/file.csv", 
           storage_options = {"config": "~/.oci/config", region="ap-sydney-1"})

Note: You must ensure that you have valid cross-region permissions before attempting to instantiate a file system in a non-home region, see the list of [valid OCI Region Identifiers](https://docs.oracle.com/en-us/iaas/Content/General/Concepts/regions.htm).